# Binary Classification Model for Disease Prediction


# Part 1: Installation of necessary libraries

In [40]:
import numpy as np
import pandas as pd
import os 
import random
#import tensorflow
from scipy.spatial import distance
import time


# Part 2: Download of patient data with corresponding disease status

In [41]:
pd.set_option("display.max_columns", None) # This ensures the full dataframe is visulised and not truncated
disease_data = pd.read_csv('Training.csv')

# Part 3 : Exploratory Data Analysis


In [42]:
disease_data['EJ'] = disease_data['EJ'].map({'A': 0, 'B': 1})
disease_data=disease_data.dropna()
disease_data.describe()

,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,BN,BP,BQ,BR,BZ,CB,CC,CD,CF,CH,CL,CR,CS,CU,CW,DA,DE,DF,DH,DI,DL,DN,DU,DV,DY,EB,EE,EG,EH,EJ,EL,EP,EU,FC,FD,FE,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
count,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000
mean,0.462860,3502.844518,118.817523,37.971167,10.196422,5.379830,0.062971,10.828981,8.718112,5382.822970,21.610622,234.075259,97.265977,1245.863961,567.230581,77.408891,0.690544,87.044188,11.053932,0.030434,1.430197,0.738126,37.345089,1.373114,27.605610,52.004387,402.969021,0.597923,0.366601,148.211760,94.641978,26.829683,1.857058,1.942715,25.875018,9.206676,2.983337,1707.554914,0.320370,0.600365,69.472692,105.822050,73.934432,73.463126,7.333246,10736.159982,10.183794,4.604619,3.746416,0.423671,20.552888,132.944954,15973.953703,31.364689,52.767917,9.448235,0.186131
std,0.395050,2281.427013,134.529098,68.540076,10.898788,2.506650,0.442135,4.408974,69.114063,3165.156480,3.589723,187.996873,95.330370,8034.942617,2198.094216,152.375697,0.271286,43.167317,13.980678,0.014783,2.032407,0.271873,17.766858,0.536955,14.390960,21.662297,316.511723,1.247109,0.113340,87.399851,29.008766,7.875949,9.538617,1.572374,18.302648,6.443720,2.023494,1733.594137,1.958098,0.490271,38.636226,71.464832,413.374400,174.616230,68.693361,11694.870394,3.030552,9.095599,53.199582,1.376450,10.072061,151.520840,20083.928101,9.879365,37.285570,10.476576,0.389568
min,0.085460,192.593280,85.200147,3.177522,8.138688,0.699861,0.025578,3.396778,1.229900,1693.624320,9.886800,72.948951,1.331155,51.216883,257.432377,12.499760,0.176874,23.387600,0.510888,0.003184,1.050225,0.069225,13.784111,0.137925,7.030640,6.906400,35.998895,0.238680,0.040995,60.232470,10.345600,6.667048,0.005518,1.743070,0.804068,4.926396,0.286201,185.594100,0.003042,0.000000,5.394675,78.526968,3.828384,7.534128,0.296850,1655.963088,3.583450,0.173229,0.497060,0.067730,4.874248,72.611063,13.038894,9.432735,0.897628,0.001129,0.000000
25%,0.246766,2198.933605,85.200147,12.030297,8.138688,4.066281,0.025578,8.532908,1.229900,4161.104110,19.773600,162.136451,27.797304,417.904995,257.432377,25.146576,0.561490,64.575542,5.059464,0.023432,1.050225,0.588375,30.258443,1.063402,7.030640,38.824735,189.447677,0.238680,0.298239,102.607074,77.987910,21.519484,0.005518,1.743070,14.373277,6.145626,1.628524,1116.429488,0.003042,0.000000,30.416100,78.526968,3.828384,25.468380,0.296850,5456.285262,8.602347,0.173229,0.552957,0.067730,14.020440,72.611063,3247.748035,25.033958,23.279172,0.150069,0.000000
50%,0.341840,3090.383560,85.200147,20.135146,8.138688,4.947752,0.025578,10.681890,1.229900,4992.779215,21.892200,196.916386,60.437303,609.091957,257.432377,44.608519,0.660746,78.725040,8.952704,0.027860,1.050225,0.737531,35.166395,1.351665,36.034882,50.207200,310.527613,0.238680,0.358023,131.295904,96.284040,25.989204,0.065522,1.743070,21.142721,8.193579,2.543561,1497.015181,0.073008,1.000000,71.311169,78.526968,23.271612,35.165928,1.695013,7528.932210,9.990934,2.151784,1.157970,0.250601,18.650510,72.611063,8762.168480,30.519642,43.785085,0.594000,0.000000
75%,0.552285,4367.771182,107.739039,37.083478,8.138688,6.183582,0.037910,13.185360,5.390476,6020.698707,24.010800,252.188707,133.849459,945.719563,257.432377,80.616559,0.772786,97.169532,13.190718,0.034477,1.242766,0.859509,40.802259,1.644066,37.940179,61.971360,508.609126,0.238680,0.426348,169.428765,110.829890,31.086732,1.058690,1.743070,32.517588,10.560475,3.807279,1912.175338

In [43]:
print(len(disease_data))
print(len(disease_data.columns))
disease_data= disease_data.reset_index(drop=True)
disease_data=disease_data.drop("Id",axis=1)
disease_data_class=disease_data["Class"]
disease_data=disease_data.drop("Class",axis=1)
print(disease_data)

548
58
           AB          AF          AH          AM         AR        AX   
0    0.209377  3109.03329   85.200147   22.394407   8.138688  0.699861  \
1    0.145282   978.76416   85.200147   36.968889   8.138688  3.632190   
2    0.470030  2635.10654   85.200147   32.360553   8.138688  6.732840   
3    0.252107  3819.65177  120.201618   77.112203   8.138688  3.685344   
4    0.380297  3733.04844   85.200147   14.103738   8.138688  3.942255   
..        ...         ...         ...         ...        ...       ...   
543  0.175193  2607.26686   85.200147    7.067354   8.138688  4.030845   
544  0.149555  3130.05946  123.763599    9.513984  13.020852  3.499305   
545  0.435846  5462.03438   85.200147   46.551007  15.973224  5.979825   
546  0.427300  2459.10720  130.138587   55.355778  10.005552  8.070549   
547  0.482849  2672.53426  546.663930  112.006102   8.138688  3.198099   

           AY         AZ          BC         BD        BN          BP   
0    0.025578   9.812214    5.5

# Part 4 : Generating New Data Features

In [44]:
def euclidean_distance(sample1, sample2):
    """Calculate the Euclidean distance between two samples."""
    return distance.euclidean(sample1, sample2)

In [45]:
def euclidean_feature_generator(Sample_List):
    euclidean_feature_dataframe=pd.DataFrame()
    
    for Sample in range(0,len(Sample_List)):#0:640
        SamplePairsData=[]

        for Alternate_Sample in range(0,len(Sample_List)):
            SamplePairsData.append(euclidean_distance(Sample_List.iloc[Sample,:],Sample_List.iloc[Alternate_Sample,:]))

        temp_df = pd.DataFrame([SamplePairsData])
        euclidean_feature_dataframe = pd.concat([euclidean_feature_dataframe, temp_df], ignore_index=False)
    return(euclidean_feature_dataframe)


In [54]:
def TrainingDataSelection(Samples,DataFrame,Euclidean_Distance_DataFrame):#The function takes two arguments: Samples, which is a list of indices representing the rows of the DataFrame to be selected, and DataFrame, which is the original DataFrame from which the data is to be selected.
    New_Euclidean_Distance_DataFrame = pd.DataFrame(np.zeros(Euclidean_Distance_DataFrame.shape), columns=Euclidean_Distance_DataFrame.columns, index=Euclidean_Distance_DataFrame.index)
    Absent=[]
    for row in range(0,len(Euclidean_Distance_DataFrame)):

        if row in Samples:
            for sampletosample in range(0,len(Euclidean_Distance_DataFrame.columns)):
                if sampletosample in Samples:
                    New_Euclidean_Distance_DataFrame.iloc[row,sampletosample]=Euclidean_Distance_DataFrame.iloc[row,sampletosample]
        else:
            Absent.append(row)

    DataFrame= DataFrame.reset_index(drop=True)
    New_Euclidean_Distance_DataFrame= New_Euclidean_Distance_DataFrame.reset_index(drop=True)
    Final_DataFrame = pd.concat([DataFrame, New_Euclidean_Distance_DataFrame], axis=1)
    Final_DataFrame=Final_DataFrame.drop(Absent)
    return(Final_DataFrame)

In [64]:
from sklearn.model_selection import train_test_split
def split_data(df, test_size=0.2, val_size=0.25, random_state=None):

    train_df, test_df = train_test_split(df, test_size=test_size, random_state=random_state)
    train_df, val_df = train_test_split(train_df, test_size=val_size, random_state=random_state)

    train_indices = train_df.index.tolist()
    val_indices = val_df.index.tolist()
    test_indices = test_df.index.tolist()

    return (train_df, train_indices), (val_df, val_indices), (test_df, test_indices)
(train_df, train_indices),(val_df, val_indices), (test_df, test_indices)= split_data(disease_data)
if 0 in train_indices:
    print("blue")

In [48]:
Euclidean_Distance_DataFrame=(euclidean_feature_generator(disease_data))
#TrainingDataSelection(train_indices,disease_data,Euclidean_Distance_DataFrame)

In [65]:

TrainingData=TrainingDataSelection(train_indices,disease_data,Euclidean_Distance_DataFrame)
TestData=TrainingDataSelection(test_indices,disease_data,Euclidean_Distance_DataFrame)

In [66]:
TrainingData.to_excel('Training Data.xlsx', index=False)
TrainingLabels=
TestData.to_excel("Test Data.xlsx",index=False)
TestLabels=
disease_data.to_excel("disease data.xlsx", index=False)

# Part 5 : Creation of a machine learning model

# Part 6 : Analysis of models efficiency and loss values

# Part 7 : Log of models